# Lab_6

## Zadania #1
**1.1 Zmodyfikować definicję tak, aby zamiast extractMaybe wykorzystać >^$>**

In [1]:
extractMaybe :: Maybe a -> a
extractMaybe Nothing  = error "Nothing inside!"
extractMaybe (Just x) = x

(>.>>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c)
f >.>> g = \x -> g (extractMaybe (f x)) -- do zmodyfikowania

(>^$>) :: Maybe a -> (a -> Maybe b) -> Maybe b
Nothing  >^$> _ = Nothing
(Just x) >^$> f = f x
infixl 1 >^$>

Line 6: Avoid lambda
Found:
\ x -> g (extractMaybe (f x))
Why not:
g . extractMaybe . f

In [2]:
(>.>>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c)
f >.>> g = \x -> f x >^$> g

(\x -> Just (x+2)) >.>> (\x -> Just (x*2)) $ 3

Just 10

**1.2 Ponownie zmodyfikować definicję, ale tym razem zamiast extractMaybe wykorzystać fmap**

In [3]:
joinMaybe :: Maybe (Maybe a) -> Maybe a
joinMaybe Nothing = Nothing
joinMaybe (Just (Just x)) = Just x
joinMaybe (Just Nothing) = Nothing

(>.>>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c)
f >.>> g = \x -> joinMaybe (fmap g (f x))

(\x -> Just (x+2)) >.>> (\x -> Just (x*2)) $ 3

Line 7: Avoid lambda
Found:
\ x -> joinMaybe (fmap g (f x))
Why not:
joinMaybe . fmap g . f

Just 10

**1.3 Przeanalizować możliwość napisania funkcji extractIO (odpowiednika extractMaybe dla IO); uwaga: rozwiązanie wykorzystujące unsafePerformIO należy pominąć**

In [4]:
-- TODO

**1.4 Napisać funkcje/operatory extract, insert, >^$> i (>.>>) dla funkcji postaci**

In [5]:
-- TODO

## Zadania #2
**2.1 (Dla monady Maybe) zdefiniować (>=>) przy pomocy >>=; czy można tę definicję uogólnić, aby była prawdziwa dla dowolnej monady?**

In [6]:
(>=>) :: Monad m => (a -> m b) -> (b -> m c) -> (a -> m c)
(>=>) f g = \x -> f x >>= g

Line 2: Redundant lambda
Found:
(>=>) f g = \ x -> f x >>= g
Why not:
(>=>) f g x = f x >>= g

**2.2 Napisać funkcję join dla monady Maybe**

In [7]:
maybeJoin :: Maybe (Maybe a) -> Maybe a
maybeJoin (Just (Just a)) = Just a
maybeJoin (Just Nothing) = Nothing
maybeJoin Nothing = Nothing

**2.3 Napisać trzy wersje funkcji:**

In [8]:
safeDiv :: Int -> Int -> Maybe Int
safeDiv x y | y /= 0    = Just $ x `div` y
            | otherwise = Nothing

safeSum10DivXi :: [Int] -> Maybe Int
safeSum10DivXi [] = Just 0
safeSum10DivXi (x:xs) = 
    case safeDiv 10 x of
        Nothing -> Nothing
        Just newX -> case safeSum10DivXi xs of
                            Nothing -> Nothing
                            Just newY -> Just (newX + newY)
                            
safeSum10DivXi [1,2,3,4]
safeSum10DivXi [1,0,3]

Just 20

Nothing

In [ ]:
safeSum10DivXi' :: [Int] -> Maybe Int
safeSum10DivXi' [] = Just 0
safeSum10DivXi' (x:xs) = do
    newX <- safeDiv 10 x
    newXs <- safeSum10DivXi' xs
    return $ newX + newXs
    
safeSum10DivXi [1,2,3,4]
safeSum10DivXi [1,0,3]

In [32]:
safeSum10DivXi'' :: [Int] -> Maybe Int
safeSum10DivXi'' [] = Just 0
safeSum10DivXi'' (x:xs) = safeDiv 10 x >>= \x -> safeSum10DivXi'' xs >>= \y -> return $ x + y

safeSum10DivXi [1,2,3,4]
safeSum10DivXi [1,0,3]

Just 20

Nothing

**2.4 Jak wyżej, ale dla funkcji**

In [34]:
--safeProduct10DivXi [Int] -> Maybe Int
--safeAbs10DivXi [Int] -> Maybe Int

-- TODO

**2.5 Sprawdzić możliwość uogólnienia powyższych rozwiązań przy pomocy funkcji**

In [35]:
--foldM :: (Foldable t, Monad m) => (b -> a -> m b) -> b -> t a -> m b
-- TODO

**2.5 Sprawdzić "prawa monadyczne" dla monady Maybe; wskazówka: wykorzystać wersję z operatorem >=>**

In [36]:
-- TODO

## Zadania #3
**3.1 Napisać funkcję join dla monady (Either e)**

In [38]:
eitherJoin :: Either a (Either a b) -> Either a b
eitherJoin (Right (Left a)) = Left a
eitherJoin (Right (Right a)) = Right a
eitherJoin (Left a) = Left a

**3.2 Napisać definicję funkcji safeF5'' (odpowiadającą tej z poprzedniego ćwiczenia, dotyczącego monady Maybe) z wykorzystaniem monady Either e**

In [41]:
-- ? jest podana literalnie wyżej
-- napiszę coś takiego
safeDiv' :: Int -> Int -> Either String Int
safeDiv' x y | y /= 0    = Right $ x `div` y
            | otherwise = Left "Cannot div by zero!"

safeF5'' :: [Int] -> Either String Int
safeF5'' [] = Right 0
safeF5'' (x:xs) = safeDiv' 10 x >>= \x -> safeF5'' xs >>= \y -> return $ x + y

safeF5'' [1,2,3,4]
safeF5'' [1,0,3]

Right 20

Left "Cannot div by zero!"

## Zadania #4
**4.1 Napisać funkcję join dla monady []**

In [42]:
joinList :: [[a]] -> [a]
joinList [] = []
joinList (xs:xss) = xs ++ joinList xss

Line 2: Use foldr
Found:
joinList [] = []
joinList (xs : xss) = xs ++ joinList xss
Why not:
joinList xss = foldr (++) [] xss